In [1]:
import pandas as pd
import plotly.offline as pyo
import plotly.graph_objs as go

In [2]:
import os.path

data = pd.read_json('https://api.covid19api.com/all')


In [3]:
data["Date"] = pd.to_datetime(data["Date"], errors='coerce').dt.date
data["CountryProvince"] = data["Country"]+data["Province"]

In [4]:
#getting start date of contagion
temp_first = data[(data['Status']=='confirmed') & (data['Cases']>0)].groupby(['CountryProvince'])['Date'].min().reset_index().rename(columns={"Date": "FirstDate"})
temp_max = data[(data['Status']=='confirmed') & (data['Cases']>=0)].groupby(['CountryProvince'])['Cases'].max().reset_index().rename(columns={"Cases": "MaxCases"})


temp_max.head()

,CountryProvince,MaxCases
0,Afghanistan,11
1,Albania,38
2,Algeria,37
3,Andorra,1
4,Antigua and Barbuda,1


In [5]:
data = data.set_index('CountryProvince')

In [6]:
#adding start date of contagion to data
data = data.join(temp_first.set_index('CountryProvince'))
data = data.join(temp_max.set_index('CountryProvince'))

In [7]:
import numpy as np
data["DaysSinceConfirmed"] = np.where(data['Date']>=data['FirstDate'], (data['Date']-data['FirstDate']).dt.days, 0)

In [8]:
data.head()

,Country,Province,Lat,Lon,Date,Cases,Status,FirstDate,MaxCases,DaysSinceConfirmed
CountryProvince,,,,,,,,,,
Afghanistan,Afghanistan,,33.0,65.0,2020-01-22,0,confirmed,2020-02-24,11,0.0
Afghanistan,Afghanistan,,33.0,65.0,2020-01-23,0,confirmed,2020-02-24,11,0.0
Afghanistan,Afghanistan,,33.0,65.0,2020-01-24,0,confirmed,2020-02-24,11,0.0
Afghanistan,Afghanistan,,33.0,65.0,2020-01-25,0,confirmed,2020-02-24,11,0.0
Afghanistan,Afghanistan,,33.0,65.0,2020-01-26,0,confirmed,2020-02-24,11,0.0


In [14]:
status = "confirmed"

temp = data[(data["MaxCases"]>=500) & (data["Status"] == status)].copy()
temp.head()

,Country,Province,Lat,Lon,Date,Cases,Status,FirstDate,MaxCases,DaysSinceConfirmed
CountryProvince,,,,,,,,,,
Austria,Austria,,47.5162,14.5501,2020-01-22,0,confirmed,2020-02-25,655,0.0
Austria,Austria,,47.5162,14.5501,2020-01-23,0,confirmed,2020-02-25,655,0.0
Austria,Austria,,47.5162,14.5501,2020-01-24,0,confirmed,2020-02-25,655,0.0
Austria,Austria,,47.5162,14.5501,2020-01-25,0,confirmed,2020-02-25,655,0.0
Austria,Austria,,47.5162,14.5501,2020-01-26,0,confirmed,2020-02-25,655,0.0


In [15]:
provinces = temp.index.unique()
plotdata = []
for province in provinces:
    # What should go inside this Scatter call?
    province_data = temp[temp.index == province]
    trace = go.Scatter(x = province_data.DaysSinceConfirmed, 
                       y = province_data.Cases, 
                       mode='lines', 
                       name = province,
                       text = province_data['Date'],
                       hoverinfo = "text+x+name+y"
                    )
    plotdata.append(trace)
    
layout = go.Layout(
    title = '(Logarithmic) Daily confirmed cases per Country/Province where more than 1000 cases have been registered',
    xaxis = dict(title = 'Day since first confirmed local case'), # x-axis label
    yaxis = dict(title = 'Confirmed cases'), # y-axis label
    yaxis_type='log'
) 

fig = go.Figure(data = plotdata, layout = layout)

fig.show()